<a href="https://colab.research.google.com/github/harlepengren/DataAnalysis/blob/main/Retirement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retirement Model
The goal of this notebook is to model retirement. In particular, trying to understand the principal needed for retirement and the strategy throughout requirement to maintain capital.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
spx = pd.read_csv("https://raw.githubusercontent.com/harlepengren/DataAnalysis/main/data/SPX_annual.csv")
spx.head()

,Date,Close,Volume,Change,LogChange
0,1/3/28,17.760000,0,0.0000,0.000000
1,1/2/29,24.809999,0,0.3970,0.000000
2,1/2/30,21.180000,0,-0.1463,-0.158189
3,1/2/31,15.850000,0,-0.2517,-0.289888
4,1/4/32,7.560000,0,-0.5230,-0.740298
